In [1]:
# findspark ...

In [2]:
# start spark session
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pv_diff_app").getOrCreate() 
spark

In [3]:
import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_columns = None 
pd.options.display.max_colwidth = None
pd.options.display.float_format = lambda x: '{:,.2f}'.format(x)
# pd.options.display.precision = 2

# as in https://towardsdatascience.com/8-commonly-used-pandas-display-options-you-should-know-a832365efa95

In [4]:
#load file
pvdf = spark.read.csv('pv_sample_input_1.csv',inferSchema=True,header=True)
pvdf.printSchema()

root
 |-- pnl report: string (nullable = true)
 |-- report ccy: string (nullable = true)
 |-- v: string (nullable = true)
 |-- deal id: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- deal ccy: string (nullable = true)
 |-- PV: double (nullable = true)



In [5]:
pvccy = pvdf.where(pvdf['report ccy'] == 'CCY')
# as in https://sparkbyexamples.com/pyspark/pyspark-where-filter/
pvccy=pvccy.withColumn("PV",pvdf["PV"].cast('float'))
# just in case:

In [6]:
pvccy.toPandas()

,pnl report,report ccy,v,deal id,product,deal ccy,PV
0,LDN desk CCY,CCY,v1,100,irs,GPB,"1,023.21"
1,LDN desk CCY,CCY,v1,101,irs,USD,"1,012.54"
2,SYD desk CCY,CCY,v1,202,repo,AUD,"1,012.98"
3,NY desk CCY,CCY,v1,303,sfut,EUR,"1,022.15"
4,LDN desk CCY,CCY,v2,100,irs,GPB,"1,023.76"
5,LDN desk CCY,CCY,v2,102,irs,NZD,"1,021.24"
6,SG desk CCY,CCY,v2,202,repo,AUD,"1,000.34"
7,NY desk CCY,CCY,v2,303,sfut,EUR,"1,003.74"


In [7]:
pvccy.groupBy('pnl report').pivot("v").sum("PV").toPandas()
# as in https://sparkbyexamples.com/pyspark/pyspark-pivot-and-unpivot-dataframe/

,pnl report,v1,v2
0,NY desk CCY,"1,022.15","1,003.74"
1,SG desk CCY,NaN,"1,000.34"
2,SYD desk CCY,"1,012.98",NaN
3,LDN desk CCY,"2,035.75","2,045.00"


In [8]:
# filtering via inner join with control table

In [9]:
scope_pddf = pd.DataFrame({
                    'product': ['irs','fra','lfut','repo'],
                    'take': [True,False,False,False]
                    })
 
scope_psdf = spark.createDataFrame(scope_pddf)

# as in https://www.geeksforgeeks.org/creating-a-pyspark-dataframe/ 

In [10]:
scope_psdf = scope_psdf.where(scope_psdf['take'])

In [11]:
scope_psdf.show()

+-------+----+
|product|take|
+-------+----+
|    irs|true|
+-------+----+



In [12]:

pvccyj=pvccy.join(other=scope_psdf,on='product',how='inner')

pvt=pvccyj.groupBy('pnl report').pivot("v").sum("PV")

In [13]:
from pyspark.sql.functions import col
pvt=pvt.withColumn("DELTA",col("v2")-col("v1"))

In [14]:
pvt.toPandas()

,pnl report,v1,v2,DELTA
0,LDN desk CCY,"2,035.75","2,045.00",9.25


In [15]:
# NOT IN: showing what I am going to miss with inner join = non -irs
# as in https://www.datasciencemadesimple.com/join-in-pyspark-merge-inner-outer-right-left-join-in-pyspark/

In [17]:
pvccy.join(other=scope_psdf,on='product',how='anti').toPandas()

,product,pnl report,report ccy,v,deal id,deal ccy,PV
0,repo,SYD desk CCY,CCY,v1,202,AUD,"1,012.98"
1,repo,SG desk CCY,CCY,v2,202,AUD,"1,000.34"
2,sfut,NY desk CCY,CCY,v1,303,EUR,"1,022.15"
3,sfut,NY desk CCY,CCY,v2,303,EUR,"1,003.74"


In [ ]:
pvccy.groupBy('pnl report').pivot("v").sum("PV").withColumn("DELTA",col("v2")-col("v1")).toPandas()

,pnl report,v1,v2,DELTA
0,NY desk CCY,"1,022.15","1,003.74",-18.41
1,SG desk CCY,NaN,"1,000.34",NaN
2,SYD desk CCY,"1,012.98",NaN,NaN
3,LDN desk CCY,"2,035.75","2,045.00",9.25
